### Dictionary Custom Classes and Hashing

#### Quick Review

How Python inserts a key/value item in a dictionary (simplified)

- Calculates hash of key 
- Mods this based on dictionary size (allocated)
- This gives start index in hash table (sequence of slots)
- Then generates probe sequence (sequence of valid indices)
- Iterates over probe sequence
 - is the slot at that index empyty?
  - if yes, store the new item there (hash, key, value)
  - if no, then continue iteration to look for an empty slot

So the more hash collisions we have, the more inefficient the dictionary is

What about the reverse? (Finding a key in a dictionary)

- Hash the key
- Mod dictionary size (allocated)
- Start index in hash table
- Generate probe sequence
- loop over probe sequence
 - is slot empty?
 - yes -> key does not exist in dictionary
 - no -> are hashes equal and keys equal (==)?
  - if yes -> found key
  - if no -> caused by hash collision, continue iteration

Again, the more hash collisions we have the more inefficient this data structure becomes

And if we have predictable hashes then the DS is subject to DOS attacks

In order for this algorithm to work:
- hash(key) when inserting item must equal hash(key) when retrieving item, otherwise we're starting search in the wrong place!
- the probe sequence must remain the same -> Python controls that, not us  
So the hash of key cannot change after storing in dict

#### Object Hashes

An object hash in Python must satisfy the following:
- must be an integer
- if a == b evaluates to True, then hash(a) == hash(b) must also evaluate to True
  - **Note:** We do not require the reverse of the above statement; two objects that are not equal can have the same hash (hash collision)

Why is this important?

Suppose we have these two tuples:

In [1]:
t1 = (1, 2)
t2 = (1, 2)

- There is no guarantee that they will have the same id
- But they are equal... t1 == t2 is True even if t1 is t2 is False

So this can occur...

In [2]:
d = {t1: 'python'}

In [3]:
d[t1]

'python'

In [4]:
d[t2]

'python'

From the dictionary's perspective t1 and t2 are the same object

#### Custom Classes

Let's say we had a custom class, Person, with a single property, name.

And we create to instances of this class each with the same name, John.

By default, custom classes compare == if they have the same id (is)

In this case, p1 == p2 will evaluate to False

By default, Python automatically makes custom objects such as the above one hashable

It uses the id to make a hash
- ids are integers and satisfy the hash rules

This is good, but not always useful....

If we put the first instance of John into a dictionary that would be fine, we would be able to look it up.

But if we tried to lookup the second instance of John in the dictionary we would get a KeyError exception, as the hash is different and the keys are not equal to each other!

So this may not be what we want

In this case we want to consider p1 and p2 as equal and be able to lookup the key using either instance

So we need to define equality for our custom class (see code for how to do this)

But once we provide an equals method in a custom class, then Pyton no longer provides a default hash based on id, so we get a TypeError if we try to hash that object

So we have to implement hashing functionality

#### Defining hash for custom class

Use the \_\_hash\_\_ special method!

When we call hash(obj) -> Python looks for \_\_hash\_\_ method on obj

*Remember that \_\_hash\_\_ must return an integer!

How do we indicate that the class is not hashable? -> Set \_\_hash\_\_ attribute to None  
\_\_hash\_\_ = None in class

- This is what happens when we define \_\_eq\_\_ , but not \_\_hash\_\_

We can however override this, by implementing the \_\_hash\_\_ method ourselves

Recall the rules:
- \_\_hash\_\_ must return an integer
- if a == b is True, then \_\_hash\_\_(a) == \_\_hash\_\_(b) is also True

**Example of above found in code**

#### Code Examples

In [5]:
t1 = (1, 2, 3)

In [6]:
t2 = (1, 2, 3)

In [7]:
id(t1), id(t2)

(1804264914840, 1804264997096)

In [8]:
t1 == t2

True

In [9]:
t1 is t2

False

In [10]:
hash(t1), hash(t2)

(2528502973977326415, 2528502973977326415)

In [11]:
d = {t1: 100}

In [12]:
d[t1]

100

In [13]:
d[t2]

100

In [14]:
d[(1, 2, 3)]

100

In [15]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'

In [16]:
p1 = Person('John', 78)
p2 = Person('John', 78)

In [17]:
p1 is p2

False

In [18]:
p1 == p2

False

In [19]:
hash(p1), hash(p2)

(-9223371924088177936, -9223371924088177940)

In [20]:
p1 = Person('John', 78)
p2 = Person('Eric', 75)
persons = {p1: 'John Object', p2: 'Eric Object'}

In [21]:
for k in persons.keys():
    print(k)

Person(name=John, age=78)
Person(name=Eric, age=75)


In [22]:
persons[Person('John', 78)]

KeyError: Person(name=John, age=78)

In [23]:
persons[p1]

'John Object'

In [26]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def __eq__(self, other):
        if isinstance(other, Person):
            return self.age == other.age and self.name == other.name
        else:
            return False

In [27]:
p1 = Person('John', 78)
p2 = Person('John', 78)

In [28]:
p1 is p2

False

In [29]:
p1 == p2

True

In [30]:
person = {p1: 'John p1'}

TypeError: unhashable type: 'Person'

In [31]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def __eq__(self, other):
        if isinstance(other, Person):
            return self.age == other.age and self.name == other.name
        else:
            return False
        
    def __hash__(self):
        return 100

In [32]:
p1 = Person('John', 78)

In [33]:
hash(p1)

100

In [34]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    __hash__ = None

In [35]:
p = Person('Eric', 75)

In [36]:
hash(p)

TypeError: unhashable type: 'Person'

In [38]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def __eq__(self, other):
        if isinstance(other, Person):
            return self.age == other.age and self.name == other.name
        else:
            return False
        
    def __hash__(self):
        return hash((self.age, self.name))

In [39]:
p1 = Person('John', 78)

In [40]:
hash(p1)

3118054141450266605

In [41]:
p1 = Person('John', 78)
p2 = Person('Eric', 75)
persons = {p1: 'John Object', p2: 'Eric Object'}

In [42]:
persons[p1]

'John Object'

In [43]:
persons[Person('John', 78)]

'John Object'

In [44]:
p2 = Person('John', 78)

In [45]:
p1 is p2

False

In [46]:
p1 == p2

True

In [47]:
hash(p2) == hash(p2)

True

In [49]:
persons[p2] is persons[p1]

True

In [50]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def __eq__(self, other):
        if isinstance(other, Person):
            return self.age == other.age and self.name == other.name
        else:
            return False
        
    def __hash__(self):
        return 'hash'

In [51]:
p1 = Person('Eric', 75)

In [52]:
hash(p1)

TypeError: __hash__ method should return an integer

In [53]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def __eq__(self, other):
        if isinstance(other, Person):
            return self.age == other.age and self.name == other.name
        else:
            return False
        
    def __hash__(self):
        return 100

In [54]:
p1 = Person('John', 78)
p2 = Person('Eric', 75)

In [55]:
hash(p1) == hash(p2)

True

In [56]:
p1 == p2

False

In [57]:
persons = {p1: 'John object', p2: 'Eric object'}

In [58]:
persons[p1]

'John object'

In [59]:
persons[Person('John', 78)]

'John object'

In [60]:
class Number:
    def __init__(self, x):
        self.x = x
        
    def __eq__(self, other):
        if isinstance(other, Number):
            return self.x == other.x
        else:
            return False
        
    def __hash__(self):
        return hash(self.x)

In [61]:
class SameHash:
    def __init__(self, x):
        self.x = x
        
    def __eq__(self, other):
        if isinstance(other, SameHash):
            return self.x == other.x
        else:
            return False
        
    def __hash__(self):
        return 100

In [64]:
numbers = {Number(i): 'some value' for i in range(1000)}
same_hashes = {SameHash(i): 'some value' for i in range(1000)}

In [65]:
numbers[Number(500)]

'some value'

In [66]:
same_hashes[SameHash(200)]

'some value'

In [67]:
from timeit import timeit

In [69]:
timeit('numbers[Number(500)]', globals=globals(), number=10_000)

0.005504300000211515

In [71]:
timeit('same_hashes[SameHash(500)]', globals=globals(), number=10_000)

0.7996972999999343

In [75]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'({self.x}, {self.y})'

In [76]:
pt = Point(1, 2)

In [77]:
print(pt)

(1, 2)


In [79]:
points = {Point(0,0): 'origin', Point(1,1): 'second pt'}

In [80]:
points

{(0, 0): 'origin', (1, 1): 'second pt'}

In [81]:
points[Point(0,0)]

KeyError: (0, 0)

In [83]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'({self.x}, {self.y})'
    
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False
        
    def __hash__(self):
        return hash((self.x, self.y))

In [84]:
points = {Point(0,0): 'origin', Point(1,1): 'second pt'}

In [85]:
points[Point(0,0)]

'origin'

In [87]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'({self.x}, {self.y})'
    
    def __eq__(self, other):
        if isinstance(other, tuple) and len(other) == 2:
            other = Point(*other)            
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False
        
    def __hash__(self):
        return hash((self.x, self.y))

In [88]:
points = {Point(0,0): 'origin', Point(1,1): 'second pt'}

In [89]:
points[Point(0,0)]

'origin'

In [90]:
points[(0,0)]

'origin'

In [91]:
pt1 = Point(0, 0)
pt2 = Point(1, 1)
points = {pt1: 'origin', p2: 'pt at (1, 1)'}

In [92]:
points[pt1], points[Point(0,0)], points[(0,0)]

('origin', 'origin', 'origin')

In [93]:
pt1.x = 10

In [94]:
pt1

(10, 0)

In [95]:
points[pt1]

KeyError: (10, 0)

In [96]:
for k, v in points.items():
    print(k, v)

(10, 0) origin
Person(name=Eric, age=75) pt at (1, 1)


In [97]:
points[(10, 0)]

KeyError: (10, 0)

In [100]:
k1 = list(points.keys())[0]

In [101]:
k1

(10, 0)

In [102]:
hash(k1)

3713074054238842681

In [103]:
hash(Point(10, 0))

3713074054238842681

In [105]:
class Person:
    def __init__(self, id, name, age):
        self._id = id
        self.name = name
        self.age = age
        
    def __repr__(self):
        return f'Person(id={self._id}, name={self.name}, age={self.age})'
    
    def __eq__(self, other):
        if isinstance(other, Person):
            return self._id == other._id
        else:
            return False
        
    def __hash__(self):
        return hash(self._id)

In [106]:
p1 = Person('john', 'John', 78)

In [107]:
persons = {p1: 'john object'}

In [108]:
persons[p1]

'john object'

In [109]:
persons[Person('john', 'John', 78)]

'john object'

In [110]:
persons[Person('john', 'yshda98', 78)]

'john object'

In [111]:
p1.name = 'Eric'
p1.age = 75

In [112]:
p1

Person(id=john, name=Eric, age=75)

In [114]:
persons[Person('john', None, None)]

'john object'